# Transfer Learning: InceptionV3

We will use a pre-trained model with InceptionV3 architecture with "imageNet" weights. 

### imports

In [1]:
import numpy as np
from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Dropout, Dense, MaxPool2D, Flatten, AvgPool2D
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt


### Generators for loading and augmenting images

In [2]:
# train_datagen = ImageDataGenerator(rescale=1./255,
#                                      shear_range=0.1,
#                                      zoom_range=0.1,
#                                      rotation_range=5,
#                                      width_shift_range=0.1,
#                                      height_shift_range=0.1,
#                                      fill_mode='nearest')
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('../data/train',
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')
test_generator = test_datagen.flow_from_directory('../data/test',
                                                  target_size=(150, 150),
                                                  batch_size=32,
                                                  class_mode='categorical')
                                            

Found 5144 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.


### Model

In [9]:
class XrayModel(tf.keras.Model):
    def __init__(self):
        super(XrayModel, self).__init__()

        self.base_model = tf.keras.applications.InceptionV3(input_shape=(150, 150, 3),
                                                            include_top=False,
                                                            weights='imagenet')
        for layer in self.base_model.layers:
            layer.trainable = False
        for layer in self.base_model.layers[-10:]:
            layer.trainable = True

        self.pool = AvgPool2D()
        self.flatten = Flatten()
        self.classifier = Dense(3, activation='softmax')
    
    def call(self, inputs):
        x = self.base_model(inputs)
        x = self.pool(x)
        x = self.flatten(x)
        outputs = self.classifier(x)
        return outputs

model = XrayModel()


## Load the saved model

In [3]:
model = tf.keras.models.load_model('../models/inceptionV3')

2022-03-29 10:19:17.526229: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Compile and train

In [10]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics='accuracy')


In [11]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > 0.90:
            self.model.stop_training = True


In [12]:
model.fit(train_generator,
         steps_per_epoch=100,
         epochs=5,
         validation_data=test_generator,
         validation_steps=3,
         callbacks=[MyCallback(), 
                    EarlyStopping(monitor='val_loss', mode='min',
                                  verbose=1, patience=5)])

Epoch 1/5
100/100 [==============================] - 95s 912ms/step - loss: 0.4785 - accuracy: 0.8311 - val_loss: 0.3944 - val_accuracy: 0.8646
Epoch 2/5
100/100 [==============================] - 86s 861ms/step - loss: 0.2314 - accuracy: 0.9132 - val_loss: 0.1905 - val_accuracy: 0.9062


## Evaluate

In [4]:
model.evaluate(test_generator)

41/41 [==============================] - 32s 742ms/step - loss: 0.2139 - accuracy: 0.9270


[0.21387270092964172, 0.9270186424255371]

## Save

In [14]:
model.save('../models/inceptionV3', save_format='tf')

2022-03-29 10:16:49.763699: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../models/inceptionV3/assets
